In [6]:
import pandas as pd

import ktrain
from ktrain import text

import importlib
importlib.reload(ktrain)
importlib.reload(ktrain.text)

<module 'ktrain.text' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\ktrain\\text\\__init__.py'>

### Read Data

In [7]:
# Read our data
train= pd.read_csv("user_tweets_train.csv")
test = pd.read_csv("user_tweets_test.csv")

train.dropna(subset=['tw_concat_processed_text'], inplace=True)
test.dropna(subset=['tw_concat_processed_text'], inplace=True)

data = train.append(test)

data_texts = data.tw_concat_processed_text
data_labels = data[['age_group', 'gender', 'age_enc', 'gender_enc']].astype(str)

from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(data_texts, data_labels, shuffle=False, test_size = 0.20)
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, shuffle=False, test_size = 0.25)

In [65]:
test.head(10)

,username,tw_concat_stemmed,age_group,tw_concat_processed_text,realname,meslek,age_enc,gender,gender_enc
0,40emrecan,Kelimelerle anlatılamayan fedakarlık ve karş...,0-30,Kelimelerle anlatılamayan fedakarlık ve kar...,emre can �o?kun,spr,0,e,0
1,AAbdullahcam,Halkbank ta yen sezon hazırlık devam ediyor H...,0-30,Halkbank’ta yeni sezon hazırlıkları devam ediy...,Abdullah ÇAM,Voleybolcu,0,e,0
2,aahmetdulundu,HUKUK FAKÜLTESİ ÖĞRENCİLERİNDEN KAZANDIRAN KAY...,0-30,HUKUK FAKÜLTESİ ÖĞRENCİLERİNDEN KAZANDIRAN KAY...,NaN,ogr,0,e,0
3,aalperates,360 TV den yen progra Çok teşekkür eder Said ...,31-40,360 TV'den yeni program! | Çok teşekkür ed...,Alper Ateş,sunucu,1,e,0
4,acelyacakiroglu,bu gün ek ben yin tek üniversite başlay insan...,0-30,bu gün ekim ben yine tekim üniversiteye başlay...,NaN,ogr,0,k,1
5,acunilicali,yen sezon bu akşa saat 20 00 de de başlıyor 2...,51-60,yeni sezonu bu akşam saat 20.00'de 'de başlıy...,acun ?l?cal?,san,3,e,0
6,ademirguckunt,Our September newsletter is out with a focus o...,51-60,Our September newsletter is out - with a focus...,Asli Demirgüç-Kunt,Ekonomist,3,k,1
7,adem_kilicci,Eğer iç sakladık saf bir gülücük kalt on sevgi...,31-40,Eğer içinizde sakladığınız saf bir gülücük ka...,Adem Kılıççı,Survivor,1,e,0
8,ahmetercanIar,Yorumcularımızın dün ver olduk tahmi tut Si 1...,41-50,Yorumcularımızın dün vermiş olduğu tahminler t...,ahmet ercanlar,gaz,2,e,0
9,ahmetycalik5,İki doz aşın ol Pazar gün ev buluşal Ahmet Ç...,0-30,"İki doz aşını ol, Pazar günü evimizde buluşalı...",ahmet �al?k,spr,0,e,0


### Explanation Load Model

In [11]:
import pickle

with open('models/gender.svc.bin', 'rb') as handle:
    model = pickle.load(handle)

with open('models/gender.vectorizer.bin', 'rb') as handle2:
    vectorizer = pickle.load(handle2)

from sklearn.pipeline import Pipeline
pipe = Pipeline([('vectorizer', vectorizer),('model', model)])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


### Explanation LIME Part

In [51]:
import eli5
from eli5.lime import TextExplainer

import numpy as np
docs = test_texts.tolist()
labels = test_labels['gender'].tolist()
doc_lengths = np.array([len(doc) for doc in docs])

In [71]:
te1 = TextExplainer(n_samples=5000, random_state=42)
te1.fit(docs[np.argsort(doc_lengths)[1]], pipe.predict_proba)
te1.show_prediction(target_names=['e', 'k'])

In [72]:
te2 = TextExplainer(n_samples=5000, random_state=42)
te2.fit(docs[np.argsort(doc_lengths)[4]], pipe.predict_proba)
te2.show_prediction(target_names=['e', 'k'])

In [73]:
te3 = TextExplainer(n_samples=5000, random_state=42)
te3.fit(docs[np.argsort(doc_lengths)[5]], pipe.predict_proba)
te3.show_prediction(target_names=['e', 'k'], )

In [69]:
te = TextExplainer(n_samples=500, random_state=42)
te.fit(docs[16], pipe.predict_proba)
te.show_prediction(target_names=['e', 'k'], )

In [67]:
print(pipe.predict([docs[16]]))
print(pipe.predict_proba([docs[16]]))
print(labels[16])

['k']
[[0.19388011 0.80611989]]
k


In [58]:
pipe.predict(docs)

array(['e', 'e', 'e', 'e', 'e', 'e', 'k', 'e', 'e', 'e', 'e', 'e', 'e',
       'e', 'e', 'e', 'k', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e',
       'e', 'e', 'e', 'e', 'e', 'k', 'k', 'e', 'k', 'k', 'k', 'k', 'e',
       'e', 'e', 'e', 'e', 'k', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'k',
       'k', 'k', 'k', 'e', 'e', 'e', 'k', 'e', 'k', 'e', 'e', 'e', 'e',
       'e', 'e', 'e', 'e', 'e', 'k', 'e', 'e', 'k', 'e', 'k', 'e', 'e',
       'k', 'e', 'k', 'e', 'e', 'e', 'e', 'e', 'e', 'k', 'k', 'e', 'e',
       'k', 'k', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'k', 'e', 'e', 'e',
       'k', 'e', 'e', 'e', 'e', 'e', 'e', 'k', 'e', 'e', 'e', 'k', 'e',
       'k', 'k', 'k', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'k', 'k', 'e',
       'e', 'e', 'k', 'e', 'k', 'k', 'e', 'e', 'k', 'e', 'e', 'e', 'e',
       'k', 'e', 'e', 'e', 'k', 'e', 'e', 'e', 'k', 'e', 'k', 'e', 'e',
       'e', 'e', 'k', 'e', 'k', 'e', 'e', 'e', 'k', 'k', 'e', 'e', 'e',
       'k', 'e', 'e', 'k', 'k', 'e', 'e', 'e', 'e', 'e', 'e', 'e